# 第一遍
图神经网络的应用领域有药物发现,物理模拟等,这篇文章分为四部分,
- 一是什么样的数据能表示成一张图,
- 二是图和别的数据有什么不同
- 三是构建gnn的模块,
- 四是搭建gnn的一个playground

# 第二遍
## 什么是图
图表示的是一系列实体和边之间的关系 <br>
<img src="屏幕截图 2025-07-15 220509.png" width="600px"><br>
为了深入表示每个节点,每条边,我们可以把节点信息,边信息和全局信息做embedding,即把这些信息存储为向量的形式<br>
<font color="red">图神经网络的核心就是:怎么样把想要的信息表示为向量,以及向量能否通过数据学习到</font><br>
### 图中三大层面问题:<br>
- 图级别:给定一张图,对该图进行分类
- 节点级别:将学员分类到两个老师的队伍中
- 边级别:通过语义分割把人物,背景拿出来,然后分析实体之间的关系
### 在图上使用机械学习的挑战是什么
- 节点,边,全局信息,连接性四种类型的信息
- 连接性通过邻接矩阵存储,邻接矩阵任意交换行列不改变节点关系
- <font color="red">Q:如何使存储高效而不受排序影响</font>
- <font color="green">A:如图,使用列表表示每个点,边,连接性相当于是把每个点,边,点与边的关系标号</font><br>
<img src="屏幕截图 2025-07-16 132433.png" width="400px">

## 图神经网络(GNN)
### 什么是GNN
- GNN是对图上所有属性进行可优化的变换,变换保证节点重新排序后结果不变(保持图的对称信息)
- GNN是"graph-in,graph-out"即输入输出都是图
### GNN构建
- 首先,对节点向量,边向量,全局向量构建一个MLP,MLP的输入输出大小相同.
- 三个MLP组成GNN的一个层.对于顶点,边,全局向量找到对应的MLP作为更新函数
- MLP对每个向量单独作用,不会影响连接性.输出后图的属性有变化,但是图的结构不改变<br>
<img src="屏幕截图 2025-07-16 134636.png" width="400px"><br>
- 由上图,经过GNN的最后一层,我们会得到一个图,在图后加一个全连接层,分类的话神经元数量是类别数,回归神经元数量是1

### 汇聚操作(pooling)
- <font color="red">Q1:如果某个节点是没有自己的属性的,如何预测该节点的属性</font>
- <font color="green">A:把节点相连的边向量拿出来,全局向量拿出来,将向量相加求和,最后经过一个节点共享的输出层得到节点预测结果</font>
<img src="屏幕截图 2025-07-16 140233.png" width="400px"><br>
- 如果没有边向量只有节点向量,则汇聚相连的节点向量从而实现对边向量的预测<br>
<img src="屏幕截图 2025-07-16 140500.png" width="400px">
- 如果没有全局向量,就把所有的节点向量加起来通过输出层,得到对全局向量的预测<br>
<img src="屏幕截图 2025-07-16 140701.png" width="400px">


<img src="屏幕截图 2025-07-16 143226.png" width="400px"><br>
- 以上是对GNN的描述,这个模型局限性很明显,就是点和边分别作MLP,没有考虑两者的联系

## 信息传递
- 内容:在更新某个节点时,将自己的向量和邻居节点的向量进行聚合操作,在传入MLP,这和cnn的卷积操作类似但是是有区别的<br>
<img src="屏幕截图 2025-07-16 144810.png" width="400px"><br>
- 下图是信息传递层的原理,从节点到边,边到节点的传递
<img src="屏幕截图 2025-07-16 145204.png" width="400px"><br>
- 边到节点和节点到边的信息传递类似于先前的预测节点(边),
- 以节点借助边的更新为例,就是汇聚的方法收集,然后通过更新函数对其进行变换,最后存储进节点向量
- 节点和边的向量大小可能不同,可以使用线性变换或者简单拼接的方法

### 添加全局表示
- 如果说图很大,连接不紧密,聚合就需要走很远的路,解决方案是添加全局表示,全局表示是一个抽象的节点,他与所有的点和边相连
- <font color="red">作者说,这个(信息传递)可以认为是特征级的注意力机制,因为他将相近的节点聚集了过来</font>
- <font color="green">特征级的注意力机制:让模型有选择的关注输入的重要部分,抑制对无关部分的关注.模拟了人类视觉系统的注意力聚焦能力</font>